In [ ]:
%pylab inline

import os
import pandas as pd
import pickle
pj = os.path.join
import json

import seaborn as sns
sns.set(style="ticks",font_scale=1.5,font='Helvetica')
sns.set_palette(sns.hls_palette(8, l=.3, s=.8))

from scipy.optimize import minimize

In [ ]:
def draw_arc(ax):
    """ 
    Draw a reference arc in a given plot
    """
    col   = 'black'
    alpha =.8
    lw    = 2

    circangs = linspace(0,2*pi,100)
    
    def arc(center,radius,angs):
        cx,cy=center
        ax.plot((cx+radius*cos(angs)),
                (cy+radius*sin(angs)),color=col,alpha=alpha,lw=lw)
        # What that factor 100 is doing in there? Converting m to cm.

    arc(params["RIGHT_ORIGIN"],params["TARGET_RADIUS"],circangs)
    arc(params["LEFT_ORIGIN"] ,params["TARGET_RADIUS"],circangs)    
    
    arc((params["ARC_BASE_X"]-params["ARC_RADIUS_1"],params["ARC_BASE_Y"]),params["ARC_RADIUS_1"],linspace(0,   pi,100))
    arc((params["ARC_BASE_X"]+params["ARC_RADIUS_2"],params["ARC_BASE_Y"]),params["ARC_RADIUS_2"],linspace(pi,2*pi,100))
    ax.set_aspect('equal')

In [ ]:
import cPickle

In [ ]:
dat = cPickle.load(open('../data/gui6_arc_04_12.14h23m11_captured.pickle27','rb'))

In [ ]:
# Also grab a parameter file while we are at it
params = json.load(open('../data/gui6_arc_04_12.14h23m11_parameters.json'))
# Convert m to cm
for quant in ["ARC_BASE_X","ARC_BASE_Y","ARC_RADIUS_1","ARC_RADIUS_2","RIGHT_ORIGIN","LEFT_ORIGIN","TARGET_RADIUS"]:
    params[quant]=100*array(params[quant])

In [ ]:
#params

In [ ]:
traj = dat[0]["trajectory"]
traj = [ (100*x,100*y) for (_,x,y) in traj ] # now in cm

In [ ]:
y,z = zip(*traj)
f,ax = subplots(1,1,figsize=(10,5))
draw_arc(ax)
ax.plot(y,z)

In [ ]:
def scale_traj(traj,f):
    """ Scales the trajectory (pairs of (x,y) coordinates) by a given factor f,
    from the starting point (first coordinate). """
    (x0,y0) = traj[0]
    transl = [ (x-x0,y-y0) for (x,y) in traj ]
    scal = [ (f*x,f*y) for (x,y) in transl ]    
    return [ (x+x0,y+y0) for (x,y) in scal ]

In [ ]:
y,z = zip(*traj)
f,ax = subplots(1,1,figsize=(10,5))
draw_arc(ax)

sx,sy = zip(*scale_traj(zip(y,z),.6))
ax.plot(sx,sy)

In [ ]:
def in_start_target(x,y):
    dleft  = sqrt(sum(pow(array([x,y])-array(params["LEFT_ORIGIN"]),2)))
    dright = sqrt(sum(pow(array([x,y])-array(params["RIGHT_ORIGIN"]),2)))
    return dleft<params["TARGET_RADIUS"] or dright<params["TARGET_RADIUS"]

def distance_to_half_arc(x,y,cx,cy,rad,upper):
    """ 
    Distance to an arc of a given radius around a given center point.
    If upper=True, then the arc is the "upper" half of the circle, that is it extends to HIGHER y than cy
    If upper=False, then the arc is the "lower" half of the circle, that is, it extends to LOWER y than cy
    """
    # First, check that we are in the good half of the circle
    goodhalf = (y>cy and upper) or (y<cy and not upper)
    
    if goodhalf:
        dfromcenter= sqrt((x-cx)**2+(y-cy)**2)
        return abs(dfromcenter-rad)

    # else...
    # In this case, the shortest distance to the arc is the shortest
    # distance to the edges of the arc
    dleftedge  = sqrt((x-(cx-rad))**2+(y-cy)**2)
    drightedge = sqrt((x-(cx+rad))**2+(y-cy)**2)
    return min([dleftedge,drightedge])

def dist_from_arc(x,y):
    """ Given a point x,y, compute the distance to the arc """
        
    # First, let's see if we are in the starting zone or target zone
    if in_start_target(x,y): return 0.
    
    dleft  = distance_to_half_arc(x,y,params["ARC_BASE_X"]+params["ARC_RADIUS_2"],params["ARC_BASE_Y"],params["ARC_RADIUS_2"],upper=False)
    dright = distance_to_half_arc(x,y,params["ARC_BASE_X"]-params["ARC_RADIUS_1"],params["ARC_BASE_Y"],params["ARC_RADIUS_1"],upper=True)
    return min([dleft,dright])


def traj_mean_dist(traj):
    """ Given a trajectory, compute the mean distance to the target arc. """
    dists = [ dist_from_arc(x,y) for (x,y) in traj ]
    return mean(dists)

In [ ]:
traj_mean_dist(traj) # for the original trajectory

In [ ]:
traj_mean_dist(scale_traj(traj,.6))

In [ ]:
def scale_dist(traj,f):
    return traj_mean_dist(scale_traj(traj,f))

In [ ]:
scale_dist(traj,1)

In [ ]:
res = minimize(lambda f : scale_dist(traj,f), 
               1, # initial guess
               method='nelder-mead', options={'xtol': 1e-8, 'disp': True})

In [ ]:
res["x"] # the optimal factor

In [ ]:
res["fun"] # the cost at that point (mean dist)

In [ ]:
x,y = zip(*traj)
f,ax = subplots(1,1,figsize=(10,5))
draw_arc(ax)
ax.plot(x,y,'-',color='blue') # original trajectory

sx,sy = zip(*scale_traj(traj,res["x"]))
ax.plot(sx,sy,'-',color='red') # original trajectory
